In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torchvision
from torchvision.io import read_image
from torchvision import models, transforms
import glob
import os
import csv
from subprocess import Popen
from torch import optim

In [2]:
csv_file = r'C:\Users\monsi\Desktop\club\lecture5\data\driving_log.csv'
with open(csv_file) as f:
    file = csv.reader(f)
    print(next(file))

['C:\\Users\\monsi\\Desktop\\club\\lecture5\\data\\IMG\\center_2021_09_02_23_44_24_449.jpg', 'C:\\Users\\monsi\\Desktop\\club\\lecture5\\data\\IMG\\left_2021_09_02_23_44_24_449.jpg', 'C:\\Users\\monsi\\Desktop\\club\\lecture5\\data\\IMG\\right_2021_09_02_23_44_24_449.jpg', '0', '0', '0', '7.104638E-06']


The reader object is like a generator, once you have iterate the values, you cannot begin a second loop to read the values again.

In [3]:
class Data(Dataset):
    
    def __init__(self, transform = None):
        super().__init__()
        csv_file = r'C:\Users\monsi\Desktop\club\lecture5\data\driving_log.csv'
        with open(csv_file,'r') as f:
            file = csv.reader(f)
            file = list(file)
            self.images = [row[0] for row in file]
            self.labels = [float(row[3]) for row in file]
            self.transform = transform
            
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        image = read_image(self.images[idx])/255.0
        label = self.labels[idx]
        if self.transform :
            image = self.transform(image)
        return image, torch.tensor(label,dtype=torch.float)

In [44]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
])

In [45]:
dataset = Data(transform= transform)

In [46]:
dataset[0][0].shape

torch.Size([3, 224, 224])

In [47]:
train_ds, test_ds, val_ds = random_split(dataset,[5621,703,703])

In [48]:
train_dl = DataLoader(train_ds, batch_size = 16, drop_last= True, shuffle = True)
test_dl = DataLoader(test_ds, batch_size = 16, drop_last= True, shuffle = False)
val_dl = DataLoader(val_ds, batch_size = 16, drop_last= True, shuffle = False)

In [49]:
for x, y in train_dl:
    print(x.shape)
    break

torch.Size([16, 3, 224, 224])


In [50]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.convn_1 = nn.Conv2d(3,64,(2,2))
        self.convn_2 = nn.Conv2d(64,128,(2,2))
        self.convn_3 = nn.Conv2d(128,256,(2,2))
        self.max_pool = nn.MaxPool2d((3,3))
        self.layer_1 = nn.Linear(12544,1000)
        self.layer_2 = nn.Linear(1000,1000)
        self.layer_3 = nn.Linear(1000,100)
        self.layer_4 = nn.Linear(100,1)
    
    def forward(self, x):
        x = self.convn_1(x)
        x = self.max_pool(x)
        x = self.convn_2(x)
        x = self.max_pool(x)
        x = self.convn_3(x)
        x = self.max_pool(x)
        x = x.view((-1,12544))
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        return x

In [51]:
net = Net()

In [56]:
net = net.to('cuda')

In [55]:
for x, y in train_dl:
    net.forward(x)
    break

In [58]:
cost = nn.MSELoss()

In [59]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)

In [60]:
epochs = 5

In [61]:
for epoch in range(1, 1 + epochs):
    net.train()
    train_cost = 0.0
    for x, y in train_dl:
        optimizer.zero_grad()
        x = x.to('cuda', dtype = torch.float)
        y = y.to('cuda', dtype = torch.float)
        pred = net(x)
        error = cost(pred.squeeze(),y)
        train_cost = train_cost + error.item()
        error.backward()
        optimizer.step()
    
    with torch.no_grad():
        net.eval()
        val_cost = 0.0
        for x, y in val_dl:
            x = x.to('cuda', dtype = torch.float)
            y = y.to('cuda', dtype = torch.float)
            pred = net(x)
            error = cost(pred.squeeze(),y)
            val_cost = val_cost + error.item()
    
    print(f'epoch : {epoch} , train_cost = {train_cost} , val_cost = {val_cost}')

torch.save(net.state_dict(),r"C:\Users\monsi\Desktop\club\lecture5\my_model_3.pt") 

epoch : 1 , train_cost = 202.45954561734106 , val_cost = 1.3084998368285596


KeyboardInterrupt: 

In [152]:
 tt = torch.tensor([[-0.0145],
        [ 0.0195],
        [-0.0017],
        [ 0.0119],
        [-0.0111],
        [-0.0172],
        [ 0.0135],
        [-0.0216],
        [-0.0026],
        [ 0.0044],
        [ 0.0076],
        [-0.0138],
        [ 0.0156],
        [-0.0333],
        [ 0.0206],
        [-0.0216]])

In [165]:
tt.squeeze()

tensor([-0.0145,  0.0195, -0.0017,  0.0119, -0.0111, -0.0172,  0.0135, -0.0216,
        -0.0026,  0.0044,  0.0076, -0.0138,  0.0156, -0.0333,  0.0206, -0.0216])